This file forecasts derailment/subreddit using BERT.

# 1. Install requirements

In [2]:
model_name = "bert-base-uncased"
test_size = 0.2
labels = [True, False]  # replace with your real labels
num_labels = len(labels)
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

# 2. Read and Prepare the Dataset

In [3]:
import pandas as pd
from convokit import Corpus, download

In [ ]:
corpus = "the corpus of emails here"

In [ ]:
import random
email = []
is_buisness = []

for convo in corpus.iter_emails():
    email.append('the email content here')
    is_buisness.append("if the email is in enron or not here")


# Zip the lists together
combined = list(zip(email, is_buisness))

# Shuffle the combined list
random.shuffle(combined)

# Unzip back into two lists
email_shuffled, is_buisness_shuffled = zip(*combined)

# Convert back to lists (optional)
email_shuffled = list(email_shuffled)
is_buisness_shuffled = list(is_buisness_shuffled)

pre_dataframe = {"email": email_shuffled, "is_buisness":is_buisness_shuffled}
df = pd.DataFrame(pre_dataframe)

In [55]:
df.head()

,SCD,derails
0,Both participants engage in a discussion with ...,False
1,The conversation begins with Speaker1 presenti...,False
2,The discussion begins with Speaker1 presenting...,True
3,The conversation begins with four speakers exp...,True
4,The conversation began with Speaker1 presentin...,True


### Label Encoder

In [34]:
from sklearn import preprocessing

In [35]:
le = preprocessing.LabelEncoder()
le.fit(df["derails"].tolist())
df['label'] = le.transform(df["derails"].tolist())

In [36]:
df.head()

,convo_id,SCD,derails,label
0,dgd3dw3,Both participants engage in a discussion with ...,False,0
1,dp22jzs,The conversation begins with Speaker1 presenti...,False,0
2,dyl0uu1,The discussion begins with Speaker1 presenting...,True,1
3,d8fk4at,The conversation begins with four speakers exp...,True,1
4,drh8oaq,The conversation began with Speaker1 presentin...,True,1


### Train/Test/Val Split

In [37]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

In [38]:
# First split off 20% for test
df_train_val, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Now split the remaining 80% into 60% train and 20% val
# 20% out of the remaining 80% is 0.25 of that chunk
df_train, df_val = train_test_split(df_train_val, test_size=0.25, random_state=42)

# Hugging Face format
dataset = DatasetDict({
    "train": Dataset.from_pandas(df_train.reset_index(drop=True)),
    "validation": Dataset.from_pandas(df_val.reset_index(drop=True)),
    "test": Dataset.from_pandas(df_test.reset_index(drop=True)),
})


### Convert to correct dataframe

In [39]:
train_dataset = dataset["train"]
val_dataset = dataset["validation"]
test_dataset = dataset["test"]


### Tokenizer

In [40]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    return tokenizer(examples["email"], truncation=True)

In [42]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/4104 [00:00<?, ? examples/s]

In [43]:
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1369 [00:00<?, ? examples/s]

# 3. Initialize Model

In [44]:
from transformers import AutoModelForSequenceClassification

In [45]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# 4. Train model

In [46]:
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np

In [47]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [48]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = metric.compute(predictions=predictions, references=labels)["accuracy"]
    return {"eval_accuracy": acc}

In [49]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
from ray import tune

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    logging_strategy="epoch",
    eval_strategy="epoch",                 # ✅ clean & modern
    save_strategy="epoch",                 # ✅ matches eval
    save_total_limit=2,                    # ✅ keeps best 2 checkpoints
    load_best_model_at_end=True,           # ✅ will restore best version
    metric_for_best_model="eval_accuracy", # ✅ or use "eval_accuracy"
    greater_is_better=True,                # ✅ for accuracy
    report_to="none",
    learning_rate=3e-5,
    per_device_train_batch_size=128,       # for speed
    per_device_eval_batch_size=256,        # speed
    fp16=True,                             # speed
    warmup_steps=500,
    weight_decay=0.05,
)

def ray_hp_space(trial):
    return {
        "learning_rate": tune.loguniform(1e-5, 5e-5),
        "weight_decay": tune.uniform(0.0, 0.3),
    }

trainer = Trainer(
    # model=model,
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]  # 👈 Add this line
)


/tmp/ipykernel_2109262/2686343222.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
def compute_objective(metrics):
    return metrics["eval_accuracy"]


Now re-generate the model with the best hyperparameters

In [ ]:
# 4. Retrain and save
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.691200,0.680469,0.589481
2,0.670000,0.651935,0.640614
3,0.646100,0.631125,0.656684
4,0.628200,0.635144,0.662527
5,0.611700,0.623476,0.660336
6,0.610000,0.629160,0.677137
7,0.591500,0.628590,0.677137
8,0.581900,0.630778,0.669832
9,0.558200,0.626109,0.672023
10,0.533500,0.664994,0.650840


TrainOutput(global_step=330, training_loss=0.612227098869555, metrics={'train_runtime': 96.9912, 'train_samples_per_second': 423.131, 'train_steps_per_second': 3.402, 'total_flos': 5240210222975040.0, 'train_loss': 0.612227098869555, 'epoch': 10.0})

In [ ]:
# trainer.save_model('BERT')

# 5. Evaluate Model

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
preds = trainer.predict(tokenized_train)
preds = np.argmax(preds[:3][0],axis=1)
GT = df_train['label'].tolist()
print(classification_report(GT,preds))

              precision    recall  f1-score   support

           0       0.71      0.70      0.71      2047
           1       0.71      0.72      0.71      2057

    accuracy                           0.71      4104
   macro avg       0.71      0.71      0.71      4104
weighted avg       0.71      0.71      0.71      4104



In [ ]:
output = trainer.predict(tokenized_test)
logits = output.predictions  # raw model output before argmax
preds = np.argmax(logits, axis=1)  # class with highest score

GT = df_val['label'].tolist()  
print(classification_report(GT, preds))


              precision    recall  f1-score   support

           0       0.51      0.49      0.50       712
           1       0.47      0.49      0.48       657

    accuracy                           0.49      1369
   macro avg       0.49      0.49      0.49      1369
weighted avg       0.49      0.49      0.49      1369



In [ ]:
trainer.evaluate()

{'eval_accuracy': 0.677136596055515,
 'eval_loss': 0.6291604042053223,
 'eval_runtime': 0.912,
 'eval_samples_per_second': 1501.049,
 'eval_steps_per_second': 6.579,
 'epoch': 10.0}